<a href="https://colab.research.google.com/github/sawa25/ConsultantIntegrator/blob/dev/%D0%97%D0%B0%D0%B3%D1%80%D1%83%D0%B7%D1%87%D0%B8%D0%BA_EXCEL_%D1%84%D0%B0%D0%B9%D0%BB%D0%BE%D0%B2_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from openpyxl import load_workbook

def read_excel_to_markdown(file_path):
    # Загружаем рабочую книгу
    wb = load_workbook(file_path)

    # Переменная для хранения содержимого в формате Markdown
    excel_markdown = ""

    for sheet_name in wb.sheetnames:
        ws = wb[sheet_name]

        # Получаем заголовки столбцов, заменяя None на пустые строки
        headers = [str(cell.value) if cell.value is not None else '' for cell in ws[1]]

        # Читаем данные
        data = []
        for row in ws.iter_rows(min_row=2, values_only=True):
            data.append(row)

        # Формируем заголовок листа
        excel_markdown += f"# {sheet_name}\n\n"

        # Преобразуем данные в формат Markdown
        excel_markdown += '| ' + ' | '.join(headers) + ' |\n'
        excel_markdown += '| ' + ' | '.join(['---'] * len(headers)) + ' |\n'

        for row in data:
            excel_markdown += '| ' + ' | '.join([str(cell) if cell is not None else '' for cell in row]) + ' |\n'

        excel_markdown += "\n"

    return excel_markdown

def markdown_to_chunks(markdown_text, chunk_size):
    # Разделение исходного текста на строки
    lines = markdown_text.strip().split('\n')

    # Переменные для хранения заголовков и данных
    headers = ""
    separator = ""
    data = []
    current_sheet = None
    chunks = []

    # Обработка строк исходного текста
    for line in lines:
        if line.startswith('# '):
            # Если начинается новый лист
            if current_sheet:
                # Если есть текущий лист, добавляем его в chunks
                for i in range(0, len(data), chunk_size):
                    chunk = data[i:i + chunk_size]
                    chunk_markdown = f"{headers}\n{separator}\n" + "\n".join(chunk) + "\n"
                    chunks.append((current_sheet, chunk_markdown))
            # Начинаем новый лист
            current_sheet = line
            headers = ""
            separator = ""
            data = []
        elif line.startswith('|') and '---' in line and not separator:
            # Если это строка разделителя
            separator = line
        elif line.startswith('|') and not headers:
            # Если это строка с заголовками
            headers = line
        elif line.startswith('|'):
            # Если это строка с данными
            data.append(line)

    # Добавляем последний лист в chunks
    if current_sheet:
        for i in range(0, len(data), chunk_size):
            chunk = data[i:i + chunk_size]
            chunk_markdown = f"{headers}\n{separator}\n" + "\n".join(chunk) + "\n"
            chunks.append((current_sheet, chunk_markdown))

    return chunks

# Укажите путь к Excel файлу
file_path = 'ИМЯ_ФАЙЛА.xlsx'

# Заданный размер чанка
chunk_size = 5 #ввести кол-во чанков на которое разбивается таблица по умолчанию 5

# Читаем файл и преобразуем его содержимое в формат Markdown
excel_markdown = read_excel_to_markdown(file_path)

# Заданный размер чанка
chunk_size = 15

# Получаем чанки в формате Markdown
chunks_excel_markdown = markdown_to_chunks(excel_markdown, chunk_size)

# Выводим результат
for sheet_name, chunk in chunks_excel_markdown:
    if chunks_excel_markdown.index((sheet_name, chunk)) == 0 or (sheet_name != chunks_excel_markdown[chunks_excel_markdown.index((sheet_name, chunk))-1][0]):
        print(sheet_name)
    print(chunk)
    print("\n" + "="*80 + "\n")
